In [ ]:
#%%

%matplotlib qt
import mne

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import localtime, strftime

from autoreject import AutoReject, get_rejection_threshold

"""
1. Read raw file
2. Band pass filter 1 to 90hz
3. Notch filter: at 50hz harmonics
4. Visual inspection of channels. Drop bads
5. Epochs of 2s (non-overlapping)
6. Autoreject Epochs
7. Manual inspection of Epochs
8. ICA 
9. Interpolate bad channels
10. Rereferenced to average
"""
# participant ID  Ex: S02; S40
id = 'S02'


##################################
#########    FOLDERS    ##########
##################################

path = '/mnt/d/Florencia/Desktop/Facultad/Tesis/TesisFlor/Analisis/S02_auditivo.fif'



#%%
raw_folder = 'raw/'
filtered_folder = 'filtered/'
epoched_folder = 'epoched/'
epochs_rej_folder = 'epochs_rejected/'
interpolated_folder = 'interpolated/'
rereferenced_folder = 'rereferenced/'
reports_fodler = 'reports/'

    
participant = f"{id}_eegmw_{round}"
#%%
report = mne.Report(title='prueba')

#%%

##################################
########   1.READ RAW   ##########
##################################

#file = path + raw_folder + participant + '.bdf'
raw = mne.io.read_raw_fif(path, preload = True, verbose = False)

channel_types = {
    'Fp1': 'eeg', 'Fp2': 'eeg', 
    'F3': 'eeg', 'F4': 'eeg', 
    'C3': 'eeg', 'C4': 'eeg', 
    'P3': 'eeg', 'P4': 'eeg', 
    'O1': 'eeg', 'O2': 'eeg', 
    'F7': 'eeg', 'F8': 'eeg', 
    'T7': 'eeg', 'T8': 'eeg', 
    'P7': 'eeg', 'P8': 'eeg', 
    'Fz': 'eeg', 'Cz': 'eeg', 
    'Pz': 'eeg', 'M1': 'eeg', 
    'M2': 'eeg', 'AFz': 'eeg', 
    'CPz': 'eeg', 'POz': 'eeg'}
raw.set_channel_types(channel_types)

#%%

### Set montage ###
#dig_montage = mne.channels.read_custom_montage('./SMARTING_standard.elp')
raw = raw.set_montage('standard_1020')

#raw.plot_sensors(show_names=True)
#plt.show()
#%%%
#report.add_raw(raw=raw, title='Raw', psd = True)


In [ ]:
##################################
########   1.READ RAW   ##########
##################################

#file = path + raw_folder + participant + '.bdf'
raw = mne.io.read_raw_fif(path, preload = True, verbose = False)

channel_types = {
    'Fp1': 'eeg', 'Fp2': 'eeg', 
    'F3': 'eeg', 'F4': 'eeg', 
    'C3': 'eeg', 'C4': 'eeg', 
    'P3': 'eeg', 'P4': 'eeg', 
    'O1': 'eeg', 'O2': 'eeg', 
    'F7': 'eeg', 'F8': 'eeg', 
    'T7': 'eeg', 'T8': 'eeg', 
    'P7': 'eeg', 'P8': 'eeg', 
    'Fz': 'eeg', 'Cz': 'eeg', 
    'Pz': 'eeg', 'M1': 'eeg', 
    'M2': 'eeg', 'AFz': 'eeg', 
    'CPz': 'eeg', 'POz': 'eeg'}
raw.set_channel_types(channel_types)

#%%

### Set montage ###
#dig_montage = mne.channels.read_custom_montage('./SMARTING_standard.elp')
raw = raw.set_montage('standard_1020')

#raw.plot_sensors(show_names=True)
#plt.show()
#%%%
#report.add_raw(raw=raw, title='Raw', psd = True)


In [ ]:
#%%
##################################
########    2.FILTERING   ########
##################################

# band filters
hpass = 1
lpass = 45
raw_filtered = raw.copy().filter(l_freq=hpass, h_freq=lpass, picks='eeg')
#%%
#notch filter
raw_filtered = raw_filtered.notch_filter(np.arange(50, 250, 50) ) 

#raw_filtered.save(path + filtered_folder + participant + '.fif', overwrite = True)

#raw_filtered.plot_psd()

#%%

In [ ]:

##################################
####   3.VISUAL INSPECTION   ######
##################################
# Filter annotations by description
filtered_annotations = mne.Annotations(onset=[],
                                        duration=[],
                                        description=[])

for ann in raw_filtered.annotations:
    if ann['description'] in ['1']:
        filtered_annotations.append(ann['onset'], ann['duration'], ann['description'])

raw_filtered.set_annotations(filtered_annotations)

events, event_id = mne.events_from_annotations(raw_filtered, event_id={'1': 1})

raw_filtered.plot(events=events,scalings='auto')
plt.show()

#report.add_raw(raw=raw_filtered, title='Filtered Raw', psd = True)


#%%

In [ ]:

##################################
#########    4.EPOCHS   ##########
#################################


tmin = -0.7
tmax = 0.5
baseline = (-.7,-.5)
epochs = mne.Epochs(raw_filtered, events = events, event_id= event_id, baseline = baseline ,
                        tmin = tmin, tmax = tmax, picks = ('eeg'), preload = True,
                        verbose = False)

#path1='/mnt/d/Florencia/Desktop/Facultad/Tesis/TesisFlor/Analisis/S02_auditivo_epochs.fif'

#epochs.save(path1, overwrite = True)
#report.add_epochs(epochs=epochs, title='Epochs')

epochs.plot(scalings = 'auto')

In [ ]:
#%%
##################################
######    REJECT EPOCHS   ########
##################################

### Autoreject algorithm for rejecting epochs ###
ar = AutoReject(thresh_method = 'bayesian_optimization', random_state=42)
epochs_clean, reject_log = ar.fit_transform(epochs, return_log=True)  
# reject = get_rejection_threshold(epochs)  

# reject_log.plot_epochs(epochs, scalings='auto')

### Manual rejection ###
epochs_clean.plot(scalings = 'auto')
plt.show()


epochs_clean.plot_drop_log()

# report.add_epochs(epochs=epochs_clean, title='Epochs clean', psd = False)
# epochs_clean.save(path + epochs_rej_folder + participant + '.fif', overwrite = True)

In [ ]:

#%%
##################################
######   ICA  ########
##################################

n_components = 0.99  # Should normally be higher, like 0.999!!
method = 'fastica'
max_iter = 512  # Should normally be higher, like 500 or even 1000!!
fit_params = dict(fastica_it=5)
random_state = 42

ica = mne.preprocessing.ICA(n_components=n_components,
                            method=method,
                            max_iter=max_iter,
#                             fit_params=fit_params,
                            random_state=random_state)

ica.fit(epochs_clean)

# create epochs based on EOG events, find EOG artifacts in the data via pattern
# matching, and exclude the EOG-related ICA components
eog_epochs = mne.preprocessing.create_eog_epochs(raw=raw_filtered, ch_name = 'Fp1')
eog_components, eog_scores = ica.find_bads_eog(
     inst=eog_epochs,
     ch_name='Fp1',  # a channel close to the eye
     threshold=1  # lower than the default threshold
)
ica.exclude = eog_components


# ica.plot_components(inst = epochs_clean,picks=range(15))

ica.plot_sources(epochs_clean, block=True, show = True, )
plt.show()

report.add_ica(ica, title = 'ICA', inst = epochs_clean)
epochs_ica = ica.apply(inst = epochs_clean)

### Manual rejection ###
epochs_ica.plot()
plt.show()



In [ ]:
#%%
##################################
######   Interpolate chs  ########
##################################
epochs_interpolate = epochs_ica.copy().interpolate_bads()
epochs_interpolate.save(path + interpolated_folder + participant + '.fif', overwrite = True)